## Imports

In [1]:
import json
import pandas as pd
import numpy as np

## Lectura input

In [2]:
path = '/Users/herminiapadialromera/ironhackb/Proyecto_final/notebooks/input_.json'
pd.set_option('display.max_colwidth', None)
df_input_json = pd.read_json('/Users/herminiapadialromera/ironhackb/Proyecto_final/notebooks/input_.json')
with open(path,'r') as f:
    df_data_course= json.loads(f.read())
columnas_explode = ['Etapa', 'Formato', 'Temática', 'Objetivos'] #columnas implicadas en el explode
df_input_json_merge = df_input_json.apply(lambda x: x.explode() if x.name in columnas_explode else x).reset_index(drop=True)
df_input_json_merge

,Provincia,Name_center,Religious,Etapa,Temática,Objetivos,Formato,Precios por taller
0,Barcelona,Greenwich,No,Primaria,Familia & Tecnología,Ofrecer formación variada,Presencial,Entre 100 y 300
1,Barcelona,Greenwich,No,Secundaria,Arte & Cultura,Ofrecer formación variada,Online,Entre 100 y 300


## Lectura del dataset de cursos 

In [5]:
data_course_df = pd.read_csv('/Users/herminiapadialromera/ironhackb/Proyecto_final/data/Datos_ampliados.csv', sep=";", encoding='utf8')
data_course_df = pd.DataFrame(data_course_df)
data_course_df['Tipo de contenido'].fillna('No innovador', inplace= True)
columnas_explode_curso = ['Etapa', 'Formato', 'Temática', 'Objetivos'] 
data_course_df = data_course_df.explode(['Etapa']).explode(['Formato']).explode(['Temática'])
data_course_df


,Título,Etapa,Formato,Nº Sesiones,Temática,Precio por taller,Provincia,Tipo de contenido
0,¡Ponte al día con la Inteligencia Artificial!,Bachillerato,Online,5,Familia & Tecnología,130,Madrid,Innovador
1,¡Ponte al día con la Inteligencia Artificial!,Infantil,Online,5,Familia & Tecnología,130,Madrid,Innovador
2,¡Ponte al día con la Inteligencia Artificial!,Primaria,Online,5,Familia & Tecnología,130,Madrid,Innovador
3,¡Ponte al día con la Inteligencia Artificial!,Secundaria,Online,5,Familia & Tecnología,130,Madrid,Innovador
4,¿Qué harías si tu hijo fuera adicto a los videojuegos?,Bachillerato,Presencial,3,Familia & Tecnología,100,Barcelona,Innovador
...,...,...,...,...,...,...,...,...
263,Violencia: Identificación y prevención con las familias,Infantil,Online,3,Educación & Crecimiento,250,Valencia,No innovador
264,Violencia: Identificación y prevención con las familias,Primaria,Presencial,3,Educación & Crecimiento,250,Valencia,No innovador
265,Violencia: Identificación y prevención con las familias,Primaria,Online,3,Educación & Crecimiento,250,Valencia,No innovador
266,Violencia: Identificación y prevención con las familias,Secundaria,Presencial,3,Educación & Crecimiento,250,Valencia,No innovador


## Merge de los dos dataframe

In [6]:
df_merge = pd.merge(df_input_json_merge, data_course_df, on=['Provincia', 'Etapa', 'Formato', 'Temática'])
df_merge = df_merge.iloc[:,1:]
df_merge

,Name_center,Religious,Etapa,Temática,Objetivos,Formato,Precios por taller,Título,Nº Sesiones,Precio por taller,Tipo de contenido
0,Greenwich,No,Primaria,Familia & Tecnología,Ofrecer formación variada,Presencial,Entre 100 y 300,¿Qué harías si tu hijo fuera adicto a los videojuegos?,3,100,Innovador
1,Greenwich,No,Primaria,Familia & Tecnología,Ofrecer formación variada,Presencial,Entre 100 y 300,Aprende a Diseñar con Canva,5,80,No innovador
2,Greenwich,No,Primaria,Familia & Tecnología,Ofrecer formación variada,Presencial,Entre 100 y 300,Creación de Animaciones Interactivas con Scratch,3,100,Innovador
3,Greenwich,No,Secundaria,Arte & Cultura,Ofrecer formación variada,Online,Entre 100 y 300,Descubre el Arte de la Improvisación Teatral,4,150,Innovador


## FILTRO

In [7]:
def sistema_recomendacion3 (df, sesiones):
    tematica_unique_list = df_merge['Temática'].unique()
    formato_unique_list = df_merge['Formato'].unique()
    etapa_unique_list = df_merge['Etapa'].unique()

    filtro_tematica_df = pd.DataFrame()
    for tematica in tematica_unique_list:
        df_tematica_temp = df_merge[(df_merge['Nº Sesiones'] <= int(f"{sesiones}")) & (df_merge['Temática'] == tematica)].iloc[:1,:]
        filtro_tematica_df = pd.concat([filtro_tematica_df, df_tematica_temp])
        #print("filtro_tematica_df ---> ", tematica)
        #display(filtro_tematica_df)

    filtro_formato_df = pd.DataFrame()
    for formato in formato_unique_list:
        df_formato = df_merge[(df_merge['Nº Sesiones'] <= int(f"{sesiones}")) & (df_merge['Formato'] == formato)].iloc[:1,:]
        filtro_formato_df = pd.concat([filtro_formato_df, df_formato])
        #print("filtro_formato_df ---> ", formato)
        #display(filtro_formato_df)

    filtro_etapa_df = pd.DataFrame()
    for etapa in etapa_unique_list:
        df_etapa = df_merge[(df_merge['Nº Sesiones'] <= int(f"{sesiones}")) & (df_merge['Etapa'] == etapa)].iloc[:1,:]
        filtro_etapa_df = pd.concat([filtro_etapa_df, df_etapa])
        #print("filtro_etapa_df ---> ", etapa)
        #display(filtro_etapa_df)

    recomendaciones_df = pd.concat([filtro_tematica_df,filtro_formato_df,filtro_etapa_df]).drop_duplicates()
    return recomendaciones_df

In [8]:
tematica_formato_etapa_df = sistema_recomendacion3 (df_merge, 5)
tematica_formato_etapa_df

,Name_center,Religious,Etapa,Temática,Objetivos,Formato,Precios por taller,Título,Nº Sesiones,Precio por taller,Tipo de contenido
0,Greenwich,No,Primaria,Familia & Tecnología,Ofrecer formación variada,Presencial,Entre 100 y 300,¿Qué harías si tu hijo fuera adicto a los videojuegos?,3,100,Innovador
3,Greenwich,No,Secundaria,Arte & Cultura,Ofrecer formación variada,Online,Entre 100 y 300,Descubre el Arte de la Improvisación Teatral,4,150,Innovador
